<img style="float: left; margin:0px 15px 15px 0px; width:120px" src="https://www.orfeo-toolbox.org/wp-content/uploads/2016/03/logo-orfeo-toolbox.png">

# OTB Guided Tour - JURSE 2019 Vannes France - May 21th
## Yannick TANGUY and David YOUSSEFI (CNES, French Space Agency)

<br>

<b> Press <span style="color:black;background:yellow">SHIFT+ENTER</span> to execute the notebook interactively cell by cell </b></div>


## Step 2 : Compute Watermask

The aim of this second exercise is to combine NDWI2 values to create a water mask
- what kind of function do we need to implement ?
- are all the pixels valid ?
- we want to maximize water extent

We are also going to use OTB pipeline, to avoid writing intermediate file : 
- the first step will be plugged as input of the second step
- only the final application will launch the pipeline, with ExecuteAndWriteOutput()


In [ ]:
import os
import otbApplication
import rasterio

import display_api
import utils

# Data directory
DATA_DIR = "data"

# Output directory
OUTPUT_DIR = "output"

# NDWI synthesis
NDWI_synthesis = os.path.join(OUTPUT_DIR,"synthesis_NDWI2.tif")


### Compute the NDWI2

Do we need to adapt the formula to deal with invalid pixels ? 

What happens with clouds ?

In [ ]:
def app_radiometricindices(images, out_dir):
    index = 0
    for im in images:
        # Here we create an application
        app = otbApplication.Registry.CreateApplication("BandMath")
        # BandMath takes a list of images as input,
        # so we have to give a Python list [ image ], or [ image1, image2, .., imageN]
        app.SetParameterStringList("il",[im])
        out = os.path.join(out_dir, os.path.basename(im.replace("_RVBPIR.tif","_NDWI2.tif")))
        app.SetParameterString("out",out)
        app.SetParameterString("exp","(im1b2-im1b4)/(im1b2+im1b4)")
        # Plug the application in the pipeline 
        # does NOT launch the pipeline yet !
        app.Execute()
        # we save the application in a list
        if (index == 0):
            tab_App = [app]
        else:
            tab_App.append(app)
        index += 1
    # the list of applications is returned
    return tab_App

### Compute watermask from a list of NDWI2 images

We are going to use a list of images to compute a watermask (0 for land, 1 for water) : 
- the inputs must be set in a list with app.AddImageToParameterInputImageList
- the expression will be use all the images im1b1, im2b1, etc.

In [ ]:
        if index==1:
            exp = exp+"im"+str(index)+"b1"
        else:
            exp = exp+", "+"im"+str(index)+"b1"
        

In [ ]:
def compute_watermask(list_app,outputfilename):
    app = otbApplication.Registry.CreateApplication("BandMath")

    index = 1
    input_list = []
    exp = "?????"
    for appli in list_app:
        app.AddImageToParameterInputImageList("il",appli.GetParameterOutputImage("out"))
        if index==1:
            exp = exp # + "some function that involves im1b1"
        else:
            exp = exp # +", "+"im"+str(index)+"b1"def compute_watermask(list_app,outputfilename):
        index += 1

    print(exp)

    app.SetParameterString("out",outputfilename)
    app.SetParameterString("exp",exp)
    app.ExecuteAndWriteOutput()

In [ ]:
# Compute radiometric indices (NDVI & NDWI2) on a stack of images
list_app = app_radiometricindices(utils.list_images(DATA_DIR,"_RVBPIR.tif"), OUTPUT_DIR)
compute_watermask(list_app, NDWI_synthesis)

In [ ]:
import rasterio
import display_api

raster = rasterio.open(NDWI_synthesis)
m, dc = display_api.rasters_on_map([raster], OUTPUT_DIR, ["NDWI synthesis"])
m

### Go to the [Step 3](./step3_segment_watermask_threshold_NDWI.ipynb)